## Compare stats for intermediate outputs `all_bus` and `shape_dissolve`

Right now, the way hqta segments are constructed and which stops are attached differ. 

Step through the rest of the process and add these comparison stats each step of the way anyway.

In [1]:
import geopandas as gpd
import pandas as pd

import B1_bus_corridors as bus_corridors
import utilities

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
combined_tiff = gpd.read_parquet(
    f"{bus_corridors.TEST_GCS_FILE_PATH}intermediate/all_bus.parquet")

diss_tiff = gpd.read_parquet(
    f"{bus_corridors.TEST_GCS_FILE_PATH}intermediate/shape_dissolve.parquet")

In [3]:
combined_eric = gpd.read_parquet(
    f"{utilities.GCS_FILE_PATH}intermediate/all_bus.parquet")

diss_eric = gpd.read_parquet(
    f"{utilities.GCS_FILE_PATH}intermediate/shape_dissolve.parquet")

In [4]:
def unique_values(df):
    for c in ["shape_id", "stop_id", "hqta_segment_id", "calitp_itp_id"]:
        print(f"# unique {c}: {df[c].nunique()}")
        
def hqta_distribution(df):
    print(df.hq_transit_corr.value_counts())

    
def area_trips_stats(df):
    area = df.geometry.area.sum().round(3)
    am_trips = df.am_max_trips.sum().astype(int)
    pm_trips = df.pm_max_trips.sum().astype(int)

    print(f"Area: {area:,}")
    print(f"AM trips sum: {am_trips:,}")
    print(f"PM trips sum: {pm_trips:,}")
    

In [5]:
def compare(eric, tiff):
    print("*************Unique Values*************")
    print("***Eric***")
    unique_values(eric)
    
    print("***Tiff***")
    unique_values(tiff)

    print("\n*************HQTA Distribution*************")
    print("***Eric***")
    hqta_distribution(eric)
    
    print("***Tiff***")
    hqta_distribution(tiff)
    
    print("\n*************HQTA Stats*************")
    print("***Eric***")
    area_trips_stats(eric[eric.hq_transit_corr==True])
    
    print("***Tiff***")
    area_trips_stats(tiff[tiff.hq_transit_corr==True])
    
    print("\n*************Not HQTA Stats*************")
    print("***Eric***")
    area_trips_stats(eric[eric.hq_transit_corr==False])
    
    print("***Tiff***")
    area_trips_stats(tiff[tiff.hq_transit_corr==False])


In [6]:
compare(combined_eric, combined_tiff)

*************Unique Values*************
***Eric***
# unique shape_id: 3010
# unique stop_id: 14799
# unique hqta_segment_id: 31494
# unique calitp_itp_id: 161
***Tiff***
# unique shape_id: 2146
# unique stop_id: 14491
# unique hqta_segment_id: 35406
# unique calitp_itp_id: 165

*************HQTA Distribution*************
***Eric***
False    26534
True      4973
Name: hq_transit_corr, dtype: int64
***Tiff***
False    25524
True      9884
Name: hq_transit_corr, dtype: int64

*************HQTA Stats*************
***Eric***
Area: 449,128,561.225
AM trips sum: 48,220
PM trips sum: 48,857
***Tiff***
Area: 1,201,949,579.684
AM trips sum: 114,876
PM trips sum: 117,336

*************Not HQTA Stats*************
***Eric***
Area: 2,185,368,656.306
AM trips sum: 52,467
PM trips sum: 52,875
***Tiff***
Area: 3,263,783,341.991
AM trips sum: 51,441
PM trips sum: 52,336


In [7]:
compare(diss_eric, diss_tiff)

*************Unique Values*************
***Eric***
# unique shape_id: 1987
# unique stop_id: 2282
# unique hqta_segment_id: 2774
# unique calitp_itp_id: 161
***Tiff***
# unique shape_id: 2128
# unique stop_id: 2949
# unique hqta_segment_id: 3811
# unique calitp_itp_id: 165

*************HQTA Distribution*************
***Eric***
False    2195
True      579
Name: hq_transit_corr, dtype: int64
***Tiff***
False    2261
True     1550
Name: hq_transit_corr, dtype: int64

*************HQTA Stats*************
***Eric***
Area: 374,360,820.841
AM trips sum: 6,706
PM trips sum: 6,855
***Tiff***
Area: 1,069,364,983.305
AM trips sum: 16,636
PM trips sum: 17,127

*************Not HQTA Stats*************
***Eric***
Area: 1,888,995,540.223
AM trips sum: 5,098
PM trips sum: 5,126
***Tiff***
Area: 2,936,207,635.885
AM trips sum: 5,122
PM trips sum: 5,185
